In [207]:
import pandas as pd
import random as r

basic = pd.read_csv('./data/battersBasic.csv')
clutch = pd.read_csv('./data/battersClutch.csv')
exp = pd.read_csv('./data/battersExp.csv')
PA = pd.read_csv('./data/battersPA.csv')
value = pd.read_csv('./data/battersValue.csv')
hit1 = pd.read_csv('./data/battersHit1.csv')
hit2 = pd.read_csv('./data/battersHit2.csv')

In [208]:
def hit_K_BB(name, outcount, base):
    playerK = exp.loc[exp["이름"] == name]['K%'] * 10
    playerBB = exp.loc[exp["이름"] == name]['BB%'] * 10
    playerhit = 1000 - playerK - playerBB
    
    selector = r.randrange(1,1001)
    
    if (selector <= int(playerK)) :
        outcount += 1
        return 'strikeout'
    elif (selector <= int(playerK + playerBB)) :
        base << 1
        base += 1
        return 'BB'
    else :
        return isHomerun(name)


In [221]:
def isHomerun(name):
    playerHit1 = hit1.loc[hit1["이름"] == name]
    playerHrNum = basic.loc[basic["이름"] == name]['홈런']
    playerInp = playerHit1['InP%']
    playerInp = playerInp.astype(float)
    playerPA = playerHit1['타석']
    playerHitNum = int(playerInp * playerPA / 100)
    playerHrRatio = playerHrNum/playerHitNum
    playerHrRatio *= 1000
    
    playerInfieldRatio = playerHit1['방향_내야%']
    playerInfieldRatio = playerInfieldRatio.astype(float)
    playerInfieldRatio *= 10
    

    
    playerOutfieldRatio = 1000 - playerInfieldRatio.values[0] - playerHrRatio.values[0]
    
    selector = r.randrange(1,1001)
    
    
    if(selector <= int(playerHrRatio)) :
        return 'homerun'
    elif(selector <= playerHrRatio.values[0] + playerInfieldRatio.values[0]):
        return 
    else:
        return outfield(name)
    

    return playerHrRatio, 100, playerOutfieldRatio, 100, playerInfieldRatio

In [241]:
def outfield(name):
    playerOutfieldBA = float(hit1.loc[hit1["이름"] == name]['타율_외'].values[0])
    
    selector = r.randrange(1,1001)
    playerOutfieldBA *= 1000
    if(selector <= playerOutfieldBA):
        return 'outfieldHit'
    else:
        return 'FO'
    

In [259]:
def outfieldFO(base, outcount,name):
    if(outcount == 2):
        outcount += 1
        return 'out'
    outcount += 1
    
    playerOutfieldCourse = float(hit2.loc[hit2["이름"] == name]['타구방향%_우'].values[0])
    playerOutfieldCourse *= 10
    
    course = 0
    selector = r.randrange(1,1001)
    
    if(selector <= playerOutfieldCourse):
        course = 1
    
    
    
    
    if(base & 4  == 4):
        base += 4
    if((base & 2 == 2) and (course)):
        base += 2
    
    return base
    

In [284]:
playerHitNum = int(basic.loc[basic["이름"] == '로하스']['안타'].values[0])

In [324]:
def outfieldHit(name, base):
    playerHitNum = int(basic.loc[basic["이름"] == name]['안타'].values[0])
    playerHitNum -= int(hit1.loc[hit1["이름"] == name]['내야안타_개수'].values[0])
    playerHitRight = float(hit2.loc[hit2["이름"] == name]['타구방향%_우'].values[0])
    playerHitLeft = float(hit2.loc[hit2["이름"] == name]['타구방향%_좌'].values[0])
    playerHitCenter = 100 - playerHitRight - playerHitLeft
    playerHitLeft *= 10
    playerHitRight *= 10
    playerHitCenter *= 10
    player2B = int(basic.loc[basic["이름"] == name]['2루타'].values[0])
    player3B = int(basic.loc[basic["이름"] == name]['3루타'].values[0])
    player1B = playerHitNum - player2B - player3B
    

    
    player1BRatio = 1000 * player1B/playerHitNum
    player2BRatio = 1000 * player2B/playerHitNum
    player3Bratio = 1000 * player3B/playerHitNum
    

    
    selector = r.randrange(1,1001)
    selector2 = r.randrange(1,1001)
    

    
    course = 0
    
    # course = 2 left, course = 1 right, course = 0 center
    
    if(selector2 <= playerHitLeft) :
        course = 2
    elif(selector2 <= playerHitLeft + playerHitRight) :
        course = 1
    else :
        course = 0
    
    
    if(selector <= int(player1BRatio)):
        base = base << 1
        #여기도 마찬가지
    elif(selector <= int(player1BRatio + player2BRatio)):
        #여기서 고민좀 해야할듯.. 2루타시 주자들 어떻게 들어올것인지
        base = base << 2
    else:
        base = base << 3
    
    return base

In [326]:
print(outfieldHit('로하스',2))

4


In [297]:
for i in range(100):
    print(hit_K_BB('로하스'))

isHomerun('로하스')

strikeout
outfield
strikeout
outfield
outfield
BB
outfield
infield
outfield
homerun
infield
outfield
homerun
strikeout
homerun
homerun
infield
BB
BB
strikeout
outfield
infield
BB
infield
infield
strikeout
outfield
outfield
outfield
infield
outfield
outfield
infield
outfield
infield
outfield
infield
homerun
infield
outfield
outfield
infield
infield
strikeout
infield
strikeout
BB
strikeout
strikeout
outfield
strikeout
outfield
infield
outfield
outfield
outfield
BB
outfield
outfield
infield
outfield
infield
infield
outfield
outfield
infield
infield
strikeout
infield
outfield
outfield
outfield
outfield
infield
infield
outfield
infield
strikeout
BB
infield
homerun
BB
outfield
infield
infield
homerun
outfield
outfield
infield
strikeout
infield
outfield
outfield
outfield
outfield
strikeout
infield
BB
outfield
outfield


/var/folders/2d/wrk6d00s6s3flm82tz6cbxmh0000gn/T/ipykernel_71529/3129326974.py:8: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if (selector <= int(playerK)) :
/var/folders/2d/wrk6d00s6s3flm82tz6cbxmh0000gn/T/ipykernel_71529/3129326974.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  elif (selector <= int(playerK + playerBB)) :
/var/folders/2d/wrk6d00s6s3flm82tz6cbxmh0000gn/T/ipykernel_71529/4286411532.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  playerHitNum = int(playerInp * playerPA / 100)
/var/folders/2d/wrk6d00s6s3flm82tz6cbxmh0000gn/T/ipykernel_71529/4286411532.py:23: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.i

'infield'